In [1]:
import pathlib
import os
import time
import libximc.highlevel as ximc
import image_focus
from ximea import xiapi
import numpy as np

ModuleNotFoundError: No module named 'libximc'

In [2]:
# Devices search
devices = ximc.enumerate_devices(ximc.EnumerateFlags.ENUMERATE_NETWORK | ximc.EnumerateFlags.ENUMERATE_PROBE)

if len(devices) == 0:
    print("The real devices were not found. A virtual device will be used.")
else:
    # Print real devices list
    print("Found {} real device(s):".format(len(devices)))
    for device in devices:
        print("  {}".format(device))

Found 3 real device(s):
  {'uri': 'xi-com:\\\\.\\COM4', 'device_serial': 5063, 'Manufacturer': 'XIMC', 'ManufacturerId': 'SM', 'ProductDescription': 'XISM-USB', 'Major': 2, 'Minor': 2, 'Release': 4, 'ControllerName': 'Axis 2', 'CtrlFlags': 1, 'PositionerName': ''}
  {'uri': 'xi-com:\\\\.\\COM5', 'device_serial': 5045, 'Manufacturer': 'XIMC', 'ManufacturerId': 'SM', 'ProductDescription': 'XISM-USB', 'Major': 2, 'Minor': 2, 'Release': 4, 'ControllerName': 'Axis 1', 'CtrlFlags': 1, 'PositionerName': ''}
  {'uri': 'xi-com:\\\\.\\COM6', 'device_serial': 5263, 'Manufacturer': 'XIMC', 'ManufacturerId': 'SM', 'ProductDescription': 'XISM-USB', 'Major': 2, 'Minor': 2, 'Release': 4, 'ControllerName': 'Axis 3', 'CtrlFlags': 1, 'PositionerName': ''}


In [3]:
device_uri_1 = r"xi-com:\\.\COM4"
device_uri_2 = r"xi-com:\\.\COM5"
device_uri_3 = r"xi-com:\\.\COM6"

In [ ]:
#create instance for first connected camera
xim_cam = xiapi.Camera(dev_id=0)

print('Opening camera...')
xim_cam.open_device()

#set exposure
xim_cam.set_exposure(9000)
print('Camera exposure was set to %i us' %xim_cam.get_exposure())
xim_cam.set_imgdataformat('XI_RGB24')

#create instance of Image to store image data and metadata
img_xim = xiapi.Image()

#start data acquisition
print('Starting data acquisition...')
xim_cam.start_acquisition()
xim_cam.enable_auto_wb()
xim_cam.set_gammaY(0.8)

xim_cam.get_image(img_xim)
xim_cam_data = img_xim.get_image_data_numpy()

Opening camera...
Camera exposure was set to 8995 us
Starting data acquisition...


In [ ]:
# location of the folder with saved images
save_location = 'C:\\sample_imgs\\'
image_file_name = 'image' 
image_file_type = '.png'
saved_image_full_path = save_location + image_file_name + image_file_type

In [6]:
axis_x = ximc.Axis(device_uri_1)
axis_y = ximc.Axis(device_uri_2)
axis_z = ximc.Axis(device_uri_3)
# To open the connection, you must manually call `open_device()` method
axis_x.open_device()
axis_y.open_device()
axis_z.open_device()

In [7]:
pos_x = axis_x.get_position()
pos_y = axis_y.get_position()
pos_z = axis_z.get_position()

In [8]:
print(pos_x.Position)
print(pos_y.Position)
print(pos_z.Position)

-24480
21792
7387


In [17]:
axis_x.command_move(pos_x.Position+3*600, 0)

In [12]:
axis_y.command_move(pos_y.Position+1307*0, 0)

In [26]:
axis_z.command_move(pos_z.Position+40, 0)

In [12]:
com_pos, av_shr, cor_max = focus_scan(axis_z, pos_z.Position, xim_cam, img_xim, 25)

Maximum Sharpness: 109.315289511494
Corresponding Position: 7349.241379310345


In [14]:
axis_z.command_move(int(cor_max), 0)

In [3]:
# generate the set of coordinates

coord_max = 19*4000 #max. shift in x direction, um
step1 = 600 #step in x direction, um
hor_per = 4000 #period in x direction, um

x_list = generate_array(coord_max, step1, hor_per) 
x_list = x_list + pos_x.Position
x_list = x_list[:-1]

y_max = 11000 #max. shift in y direction, um
y_step = 1307 #step in y direction, um
y_list = np.arange(pos_y.Position, pos_y.Position+(y_max), y_step)  

print('Array of x-coord.:', x_list)  
print('Array of y-coord.:',y_list)   
 
arr_step = 1  
p = 0

increment_val = 988 # index of the first saved image
in_f1 = pos_z.Position
in_f = pos_z.Position
coeff_y = 400/746
temp_y = pos_y.Position

NameError: name 'pos_x' is not defined

In [4]:
x_list

array([    0,   600,  1200,  1800,  4000,  4600,  5200,  5800,  8000,
        8600,  9200,  9800, 12000, 12600, 13200, 13800, 16000, 16600,
       17200, 17800, 20000, 20600, 21200, 21800, 24000, 24600, 25200,
       25800, 28000, 28600, 29200, 29800, 32000, 32600, 33200, 33800,
       36000, 36600, 37200, 37800, 40000, 40600, 41200, 41800, 44000,
       44600, 45200, 45800, 48000, 48600, 49200, 49800, 52000, 52600,
       53200, 53800, 56000, 56600, 57200, 57800, 60000, 60600, 61200,
       61800, 64000, 64600, 65200, 65800, 68000, 68600, 69200, 69800,
       72000, 72600, 73200, 73800, 76000])

In [ ]:
for i in y_list: 
# for i in range(5):

    t = 1
    temp_y = int(i)
    # if i == 0:
    #     temp_y = temp_y
    # else:
    #     temp_y = temp_y - y_step
    axis_y.command_move(int(temp_y), 0) 
    axis_y.command_wait_for_stop(10)
        
    # optimize y-coordinate to keep the track in the middle of the field of view
    diff = conv_gauss(xim_cam, img_xim)*coeff_y
    if np.abs(diff) < 100:
        temp_y = diff + temp_y
    else:
        temp_y = temp_y

    print(int(temp_y))
    axis_y.command_move(int(temp_y), 0)
    axis_y.command_wait_for_stop(10)

    # optimize focus
    _, _, in_f1 = focus_scan(axis_z, int(in_f1), xim_cam, img_xim, 150)
    axis_z.command_move(int(in_f1), 0)
    axis_z.command_wait_for_stop(10)
    in_f = in_f1


    for j in x_list[::arr_step]: 
        axis_x.command_move(int(j), 0) 

        # 0, 600, 1200, 1800, 3970 - 1800 ...
        # if np.mod(t, 5) == 0:
        #     time.sleep((3970 - 1800)/speed_x+0.5)    
        # else:
        #     time.sleep(step1/speed_x+0.5)  

        axis_x.command_wait_for_stop(10)

        t +=1
        
        # optimize y-coordinate to keep the track in the middle of the field of view
        diff = conv_gauss(xim_cam, img_xim)*coeff_y
        if np.abs(diff) < 100:
            temp_y = diff + temp_y
        else:
            temp_y = temp_y

        print(int(temp_y))
        axis_y.command_move(int(temp_y), 0)
        axis_y.command_wait_for_stop(10)

        # optimize focus
        _, _, in_f = focus_scan(axis_z, int(in_f), xim_cam, img_xim, 25)
        axis_z.command_move(int(in_f+3*0), 0)
        axis_z.command_wait_for_stop(10)


        # save image
        xim_cam.get_image(img_xim)
        xim_cam_data = img_xim.get_image_data_numpy()
        print('Saving image...')
        y_cr = i-pos_y.Position
        x_cr = j-pos_x.Position
        cv2.imwrite((save_location + str(increment_val) + '_' + image_file_name + '_' +str(np.abs(y_cr+13*y_step)) + '_'+ str (np.abs(x_cr))+ image_file_type), xim_cam_data) 
        increment_val += 1
        print(i-pos_y.Position,j-pos_x.Position)

    axis_x.command_move(pos_x.Position, 0) 
    
    axis_x.command_wait_for_stop(10)

    p += 1 
    # arr_step*=-1 # change to -1 if snake-like array

21793
Maximum Sharpness: 78.02291015325672
Corresponding Position: 7381.827586206897
21795
Maximum Sharpness: 87.39857758620676
Corresponding Position: 7397.379310344828
Saving image...
0 0
21796
Maximum Sharpness: 85.86077394636055
Corresponding Position: 7385.793103448276
Saving image...
0 600
21795
Maximum Sharpness: 90.59548579182632
Corresponding Position: 7391.0344827586205
Saving image...
0 1200
21794
Maximum Sharpness: 89.60481226053636
Corresponding Position: 7386.689655172414
Saving image...
0 1800
21795
Maximum Sharpness: 90.79796455938697
Corresponding Position: 7381.689655172414
Saving image...
0 4000
21796
Maximum Sharpness: 86.33771743295011
Corresponding Position: 7390.482758620689
Saving image...
0 4600
21796
Maximum Sharpness: 93.21051883780342
Corresponding Position: 7380.517241379311
Saving image...
0 5200
21796
Maximum Sharpness: 88.58034259259259
Corresponding Position: 7384.310344827586
Saving image...
0 5800
21794
Maximum Sharpness: 91.94270330459769
Correspondi

KeyboardInterrupt: 

In [ ]:
print ('Stopping acquisition...')
xim_cam.stop_acquisition()

    #stop communication
xim_cam.close_device()

print ('All finished!')

In [ ]:
xim_cam.get_image(img_xim)
xim_cam_data = img_xim.get_image_data_numpy()
cv2.imwrite((save_location + str(increment_val) + '_' + image_file_name + '_' +str(np.abs(z_cr-44940-1090*3)) + '_'+ str (np.abs(x_cr))+ image_file_type), xim_cam_data) 

True

In [2]:
from scipy.interpolate import interp1d
import cv2
import numpy as np
from ximea import xiapi
import time
import scipy.optimize as opt
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

def focus_scan(axis, start, xim_cam, img_xim, z_step):

    """
    Performs a bi-directional focus scan to find the position of maximum sharpness.

    Parameters:
    axis: Motion control object for the axis.
    start (float): Starting position for the scan.
    xim_cam: Camera interface object for capturing images.
    img_xim: Image buffer used with the camera.
    z_step (float): Distance to move in each direction from the start position.

    Returns:
    common_positions (np.ndarray): Interpolated grid of positions.
    sharpness_sum (np.ndarray): Summed sharpness profile over both scans.
    max_position (float): Position corresponding to maximum sharpness.
    """
    
    # Move to the starting position minus z_step and wait for stop
    axis.command_move(start-z_step, 0)
    axis.command_wait_for_stop(10)
    
    # Define a common grid for interpolation
    common_positions = np.linspace(start-z_step, start+z_step, 30) 

    # Initialize accumulator for sharpness values
    sharpness_sum = np.zeros_like(common_positions)

    fl = 1
    
    # Perform two scans: one forward, one reverse
    for i in range(2):

        axis.command_move(start + fl*z_step, 0)
        pos, sharps = axis_motion_loop(axis, img_xim, xim_cam)
        
        interp_func = interp1d(pos, sharps, kind='linear', bounds_error=False, fill_value=0)
        sharpness_sum += interp_func(common_positions)
        # plt.plot(common_positions, sharpness_sum,'o')
    
        fl*= -1


    # Compute the average sharpness
    sharpness_avg = sharpness_sum / 2

    # Find the maximum sharpness and its corresponding position
    max_index = np.argmax(sharpness_avg)
    max_position = common_positions[max_index]
    max_sharpness = sharpness_avg[max_index]
    # plt.plot(common_positions, sharpness_avg,'o')

    # Print results
    print(f"Maximum Sharpness: {max_sharpness}")
    print(f"Corresponding Position: {max_position}")

    return common_positions, sharpness_sum, max_position


def axis_motion_loop(axis, img_xim, xim_cam):

    """
    Monitors the motion of a stage and captures image sharpness during motion.

    Parameters:
    axis: An object representing the motion axis with methods:
        - get_status(): returns motion status
        - get_position(): returns current position (with attribute `.Position`)
    img_xim: An image buffer object, used to store captured frames from the camera.
    xim_cam: A camera interface object capable of capturing images into `img_xim`.

    Returns:
    positions (np.ndarray): List of axis positions recorded during the motion.
    sharp_vals (np.ndarray): Corresponding sharpness values calculated from captured images.
    """

    positions = [] # List to store positions of the axis
    # timestamps_s = []
    sharp_vals = [] # List to store sharpness values of images at each position
    
    # t_start_s = time.time()

    # Check axis motion status in a loop
    status = axis.get_status()
    while status.MvCmdSts & ximc.MvcmdStatus.MVCMD_RUNNING:

        # timestamps_s.append(time.time() - t_start_s)
        # print('Current position:', position.Position)
        # time.sleep(0.01)

        # Capture image from the camera
        xim_cam.get_image(img_xim)
        image = img_xim.get_image_data_numpy()

        position = axis.get_position()
        positions.append(position.Position)

        # Calculate and store the sharpness of the current image
        srp = calculate_sharpness(image)
        sharp_vals.append(srp)
        # Update status for the next check of the loop condition
        status = axis.get_status()
        
    return np.array(positions), np.array(sharp_vals)
 

def generate_array(coord_max, step1, hor_per):

    """
    Generates a 1D NumPy array of coordinates where:
    - Every full cycle spans `hor_per` units.
    - Within each cycle, there are 3 points spaced by `step1`.
    - The 4th point is placed to reach the full `hor_per` spacing from the start of the cycle.

    Parameters:
    coord_max (float): The maximum value the array should not exceed.
    step1 (float): Step size between the first three points in a cycle.
    hor_per (float): Total horizontal period of one full cycle.

    Returns:
    np.ndarray: Array of generated coordinate values.
    """

    result = [0] # Start with initial point at 0
    current_value = 0
    
    while current_value < coord_max:
        # Repeat step1 three times
        for _ in range(3):
            current_value += step1
            if current_value > coord_max:
                break
            result.append(current_value)
        
        # Compute and apply step2
        step2 = hor_per - 3 * step1
        current_value += step2
        if current_value > coord_max:
            break
        result.append(current_value)
    
    return np.array(result)

def conv_gauss(xim_cam, img_xim):
    
    """
    Processes an image from a camera interface to extract a smoothed intensity profile
    and determine a position of the profiles extrema.

    Parameters:
    xim_cam: An object providing access to the camera or image source.
    img_xim: An image buffer object which can be filled by the camera and converted to a NumPy array.

    Returns:
    minp (int): A position value calculated based on the smoothed intensity profile.
    """

    xim_cam.get_image(img_xim)
    image = img_xim.get_image_data_numpy()
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale

    # Compute mean intensity along the second dimension (columns)
    mean_profile = np.mean(image, axis=1)

    # Apply Gaussian filter for smoothing
    smoothed_profile = gaussian_filter1d(mean_profile, sigma=45)

    # Plot the profile
    # plt.figure(figsize=(8, 4))
    if np.abs(np.max(smoothed_profile) - np.min(smoothed_profile))/np.min(smoothed_profile) > 0.4:
        
        if np.abs(512 - np.argmin(smoothed_profile)) < 250:
            minp = 512 - np.argmin(smoothed_profile)
            clr = 'b'
        else:
            minp = 512 - np.argmax(smoothed_profile)
            clr = 'g'

    else:
        clr = 'red'
        minp = 0

    return minp

    
    # plt.plot(smoothed_profile, label=f"Smoothed Profile ({image_file}), ({minp})", linewidth=2, color = clr)
    # plt.xlabel("Column Index")
    # plt.ylabel("Mean Intensity")
    # plt.title(f"Mean Intensity Profile - {image_file}")
    # plt.legend()
    # plt.show()


def calculate_sharpness(image):

    """
    Estimates image sharpness by computing the mean gradient magnitude in a cropped region.
    The gradients are computed using the Sobel filter in both x and y directions.
    A higher mean value indicates more pronounced edges, suggesting a sharper image.
    
    Parameters:
    image (np.ndarray): Input image (assumed to be in BGR format).

    Returns:
    res: Mean edge sharpness.
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale

    gray = gray[512-50:512+50, 840-90:840+90]
    # plt.imshow(gray, cmap="gray")
    # plt.show()
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)
    abs_sobel_x = cv2.convertScaleAbs(sobelx)
    abs_sobel_y = cv2.convertScaleAbs(sobely)
    dst = cv2.addWeighted(abs_sobel_x, 0.5, abs_sobel_y, 0.5, 0)
    res = np.asarray(dst).mean()
    # print(res)
    return res 


